In [ ]:

# 중도탈락학생(신입생)비율(%) 데이터 전처리 템플릿

import pandas as pd

# 📂 파일 경로 설정
file_path = "파일명을 여기에 입력하세요.xlsx"

# ✅ 데이터 로딩 (앞쪽 헤더 4줄은 무시)
df = pd.read_excel(file_path, skiprows=4)

# ✅ 연도 및 학교명, 중도탈락(신입생)비율 추출
df_filtered = pd.DataFrame({
    "기준연도": df["Unnamed: 0"].fillna(method='ffill'),
    "학교명": df["Unnamed: 5"],
    "중도탈락학생(신입생)비율(%)": df["Unnamed: 27"]
})

# ✅ 결측치 제거
df_filtered.dropna(subset=["기준연도", "학교명", "중도탈락학생(신입생)비율(%)"], inplace=True)

# ✅ 연도 정수형 변환
df_filtered["기준연도"] = df_filtered["기준연도"].astype(int)

# ✅ 저장
df_filtered.to_csv("중도탈락학생_신입생비율_출력파일.csv", index=False, encoding="utf-8-sig")

# ✅ 결과 확인
df_filtered.head()
